# 载入库和头文件，并添加上级目录

In [1]:
import sys
import os
print(os.getcwd())
sys.path.append('/home/xilinx/jupyter_notebooks/summer/modules/')  # 添加上级目录到路径
import set_lmclks
from pynq import Overlay
from rfsoc_book.helper_functions import scatterplot
import shlex
import argparse
import configs as cfgs
import parse_args
import receiver as rx
import transmitter as tx
import threading
import tx_thread
import plt_funcs as pf
import data_func as df
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go
from threading import Thread
import time


/home/xilinx/jupyter_notebooks


# 载入参数、配置时钟、载入硬件
1、从arg_configs.txt中读取硬件参数

2、通过set_lmclks.set_custom_lmclks()进行时钟配置。目前使用的时钟文件位于/modules/xrfclk中

3、向FPGA中载入硬件：ol = Overlay(ofdm_cfgs.RTL_BITFILE) RTL_BITFILE在/home/xilinx/work/summer/bitfile/目录下，这有几个bitfile，每个bitfile都如目录名所示。 *注意！在载入硬件后，发射机就会开始不断向DAC发射信号，若要接前端，注意功率问题！*

In [2]:
args = parse_args.parse_args_from_file("/home/xilinx/jupyter_notebooks/summer/modules/arg_configs.txt")
ofdm_cfgs = cfgs.ofdm_config(args)
if(ofdm_cfgs.init_ok != 1):
    print("bad init")
    sys.exit()
ofdm_cfgs.display()
import sys
import os
print(os.getcwd())
sys.path.append('/home/xilinx/jupyter_notebooks/summer/modules/')  # 添加上级目录到路径
import set_lmclks
from pynq import Overlay
set_lmclks.set_custom_lmclks()
ol = Overlay(ofdm_cfgs.RTL_BITFILE)

Parsed args from file: 'Namespace(EXE_MODE='RTL_MODE', N=64, QAM_MODE=4, N_OFDM=4, PILOT_MODE='ZEROS', N_FRAMES=100, PEAK=None, SNR=30, TX_INPUT_QAM_FILE='/home/xilinx/jupyter_notebooks/summer/dats/rtl_input/tx_input_qam.txt', TX_OUTPUT_QAM_NO_PILOT_FILE='/home/xilinx/jupyter_notebooks/summer/dats/rtl_input/tx_output_qam_no_pilot.txt', TX_OUTPUT_QAM_PILOT_FILE='/home/xilinx/jupyter_notebooks/summer/dats/rtl_input/tx_output_qam_pilot.txt', TX_OUTPUT_SINGLE_FRAME_FILE='/home/xilinx/jupyter_notebooks/summer/dats/rtl_input/single_frame.txt', TX_OUTPUT_FILE='/home/xilinx/jupyter_notebooks/summer/dats/rtl_input/rtl_test_input.txt', CHANNEL_OUTPUT_FILE=None, RTL_OUTPUT_I_FILE='/home/xilinx/work/test_AXI/2024_2_26test/rtl_output_I.txt', RTL_OUTPUT_Q_FILE='/home/xilinx/work/test_AXI/2024_2_26test/rtl_output_Q.txt', RTL_OUTPUT_COMPLEX_FILE=None, DATS_DIR='/home/xilinx/jupyter_notebooks/summer/dats/rx_output/', RTL_BITFILE='/home/xilinx/work/summer/bitfile/test_fixed_tx_16QAM_2Ghz/summer_design_w

# 生成发射数据（若只进行接收测试，不需要运行）
会根据输入的参数，生成发射数据，并保存在/dats/rtl_input中。其中，rtl_test_input.txt可以直接用于生成coe file

In [ ]:
tx0 = tx.tx(ofdm_cfgs)
tx0.tx_run()

# 初始化DMA

In [3]:
import numpy as np
from pynq import allocate
import dma_opts 

rx_I_dma = ol.axi_dma_0
rx_Q_dma = ol.axi_dma_1

print("rx_Q_dma._offset = ",rx_Q_dma.recvchannel._offset)


#input_buffer = dma_opts.init_tx_mem(ofdm_cfgs.TX_OUTPUT_FILE)
output_I_buffer = dma_opts.init_rx_mem(3200000) # the input is KB
output_Q_buffer = dma_opts.init_rx_mem(3200000) # the input is KB

rx_Q_dma._offset =  48
rx mem init complete
rx mem init complete


# 第一次时间同步，看信号是否正确

In [4]:
dma_opts.rx_dma_transfer_once(rx_I_dma,output_I_buffer)
dma_opts.rx_dma_transfer_once(rx_Q_dma,output_Q_buffer)
df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_I_FILE,output_I_buffer[0:2000])
df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_Q_FILE,output_Q_buffer[0:2000])
rx0 = rx.rx(ofdm_cfgs)
rx0.first_time_sync()

def display_real_time_plot(rx_instance, update_interval=1):
    """
    显示 RX 类实例的 time_sync_res_before_cutting 数组的实时更新图表。

    参数：
    rx_instance (RX): RX 类的实例
    update_interval (int): 更新间隔时间（秒）
    """

    # 初始化图表
    fig = go.FigureWidget()
    scatter = fig.add_scatter(y=rx_instance.time_sync_res_before_cutting, mode='lines+markers')
    fig.update_layout(
        title='Real-time Array Plot',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
    )

    def update_plot():
        dma_opts.rx_dma_transfer_once(rx_I_dma, output_I_buffer)
        dma_opts.rx_dma_transfer_once(rx_Q_dma, output_Q_buffer)
        df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_I_FILE, output_I_buffer[0:2000])
        df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_Q_FILE, output_Q_buffer[0:2000])

        rx_instance.reload_received_sig()
        rx_instance.first_time_sync()  # 更新数组
        with fig.batch_update():
            scatter.data[0].y = rx_instance.time_sync_res_before_cutting
            scatter.data[0].x = list(range(len(rx_instance.time_sync_res_before_cutting)))
            fig.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
            )

    # 创建开始按钮
    button = widgets.Button(description='Update Plot')

    # 定义按钮点击事件
    def on_button_click(b):
        update_plot()

    button.on_click(on_button_click)
    
    # 使用 VBox 组件显示按钮和绘图区域
    vbox = widgets.VBox([button, fig])
    display(vbox)

display_real_time_plot(rx0, update_interval=1)

rx dma transferring once
rx dma transfer once complete
rx dma transferring once
rx dma transfer once complete
(array([], dtype=int64), {'peak_heights': array([], dtype=float64)})


    'data': [{'mode': 'l…

# 输入截断大小和峰

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
# 创建文本输出区域
output = widgets.Output()

# 创建文本输入框
start_index_input = widgets.Text(value='', description='start_index_input:', disabled=False)
end_index_input = widgets.Text(value='', description='end_index_input:', disabled=False)
peak_height_input = widgets.Text(value='', description='peak_height_input:', disabled=False)
# 创建按钮
button = widgets.Button(description='read')

# 定义按钮点击的处理函数
def on_button_click(b):
    with output:
        clear_output(wait=True)  # 清空之前的输出
        print(f'你输入的内容是: start_index =  {start_index_input.value}, end_index = {end_index_input.value}, peak_height = {peak_height_input.value}')
    
    start_index_int = int(start_index_input.value)
    end_index_int = int(end_index_input.value)
    peak_height_int = int(float(peak_height_input.value))
    
    print("start_index_int = ",start_index_int,"end_index_int",end_index_int,"peak_height_int",peak_height_int)
    
    rx0.cutting_recv_sig(start_index_int,end_index_int,peak_height_int)
    
    start_index_input.value = ''  # 重置文本框内容
    end_index_input.value = ''  # 重置文本框内容
    peak_height_input.value = ''  # 重置文本框内容
    

# 绑定事件处理函数到按钮
button.on_click(on_button_click)

# 显示控件
display(start_index_input,end_index_input,peak_height_input, button, output)

Text(value='', description='start_index_input:')

Text(value='', description='end_index_input:')

Text(value='', description='peak_height_input:')

Button(description='read', style=ButtonStyle())

Output()

start_index_int =  0 end_index_int 2000 peak_height_int 200000000000000000


# 运行就行

In [6]:
rx0.second_time_sync()
rx0.disassemble()
rx0.channel_est(1)
rx0.demode_payload()
rx0.channel_equ(1)


(array([ 397,  461, 1037, 1101, 1677, 1741]), {'peak_heights': array([2.35176502e+17, 2.44194106e+17, 2.51211465e+17, 2.45827557e+17,
       2.36925563e+17, 2.41709890e+17])})
num_frames =  3



 in 0 frames:
 LTF start index =  333 
training_seq_start_index =  493 
payload_start_index = 653 






 in 1 frames:
 LTF start index =  973 
training_seq_start_index =  1133 
payload_start_index = 1293 



last frame not complete
i= 0 est_res: [-1.3809204e-03+0.j          1.7869273e-02+0.00158661j
  1.9720716e-02-0.00103165j -1.8378788e-03-0.00013562j
 -8.2469755e-04-0.00062758j  1.7146228e-02+0.00165115j
  1.8640051e-02-0.00065382j  1.4264307e-02+0.00182167j
 -7.7612244e-04+0.00178962j -2.1971648e-03+0.00134448j
  4.1904565e-04+0.00116154j  1.5281004e-02+0.00319889j
  1.6891230e-02+0.00029125j  1.6658392e-02-0.00164141j
  1.2674063e-02+0.00464321j  6.5289751e-05-0.00100772j
  1.3511658e-02-0.00364685j  1.4345311e-02+0.002492j
  3.4572739e-05+0.00025651j  1.0613734e-03+0.00152576j
  1.071594

# 测试
六张图从左上到右下，分别是：时域、时间同步、信道相位、信道增益、均衡前星座图和均衡后星座图

输入截断大小和peak后，然后摁一次按钮会进行一次采样和分析，在最下面的save frame按钮会将这一帧的数据保存到rtl_output中

定位的图目前还没画，等待增加

In [8]:
def display_real_time_plots(rx_instance, update_interval=1):
    """
    显示 RX 类实例的 time_sync_res_before_cutting 数组和星座图的实时更新图表。

    参数：
    rx_instance (RX): RX 类的实例
    update_interval (int): 更新间隔时间（秒）
    """

    # 初始化图表
    fig_recv_sig = go.FigureWidget()
    scatter_recv_sig = fig_recv_sig.add_scatter(y=rx_instance.recv_sig_after_cutting.real, mode='lines+markers')
    fig_recv_sig.update_layout(
        title='recv sig ',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.recv_sig_after_cutting.real) * 1.1])
    )
    
    fig_time_sync = go.FigureWidget()
    scatter_time_sync = fig_time_sync.add_scatter(y=rx_instance.time_sync_res_before_cutting, mode='lines+markers')
    fig_time_sync.update_layout(
        title='Time Sync Results Before Cutting',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
    )
    
    fig_channel_phase = go.FigureWidget()
    scatter_channel_phase = fig_channel_phase.add_scatter(y=rx_instance.channel_est_phase[0], mode='lines+markers')
    fig_channel_phase.update_layout(
        title='Channel Phase',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.channel_est_phase[0]) * 1.1])
    )
    
    fig_channel_gain = go.FigureWidget()
    scatter_channel_gain = fig_channel_gain.add_scatter(y=rx_instance.channel_est_gain[0], mode='lines+markers')
    fig_channel_gain.update_layout(
        title='Channel Gain',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.channel_est_gain[0]) * 1.1])
    )
    
    fig_constellation_no_equ = go.FigureWidget()
    scatter_constellation_no_equ = fig_constellation_no_equ.add_scatter(x=rx_instance.qam_symbs_after_ofdm_demode_1d.real, y=rx_instance.qam_symbs_after_ofdm_demode_1d.imag, mode='markers')
    fig_constellation_no_equ.update_layout(
        title='Constellation before equ',
        xaxis_title='In-Phase',
        yaxis_title='Quadrature',
        xaxis=dict(range=[-2, 2],showticklabels=False),
        yaxis=dict(range=[-2, 2],showticklabels=False)
    )

    fig_constellation_after_equ = go.FigureWidget()
    scatter_constellation_after_equ = fig_constellation_after_equ.add_scatter(x=rx_instance.demode_res_after_channel_equ_1d.real, y=rx_instance.demode_res_after_channel_equ_1d.imag, mode='markers')
    fig_constellation_after_equ.update_layout(
        title='Constellation after equ',
        xaxis_title='In-Phase',
        yaxis_title='Quadrature',
        xaxis=dict(range=[-2, 2],showticklabels=False),
        yaxis=dict(range=[-2, 2],showticklabels=False)
    )
    
    '''
    fig_ranging_res = go.FigureWidget()
    scatter_ranging_res = fig_ranging_res.add_scatter(y=rx_instance.ranging_res[0][0], mode='lines+markers')
    fig_ranging_res.update_layout(
        title='Time Sync Results Before Cutting',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.ranging_res[0][0]) * 1.1])
    )
    '''

    def update_plots(start_index_int,end_index_int,peak_height_int):
        dma_opts.rx_dma_transfer_once(rx_I_dma, output_I_buffer)
        dma_opts.rx_dma_transfer_once(rx_Q_dma, output_Q_buffer)
        df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_I_FILE, output_I_buffer[0:2000])
        df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_Q_FILE, output_Q_buffer[0:2000])

        rx_instance.reload_received_sig()
        
        rx_instance.first_time_sync()  # 更新数组
        
        rx_instance.cutting_recv_sig(start_index_int, end_index_int, peak_height_int)
        rx_instance.second_time_sync()
        rx_instance.disassemble()
        rx_instance.channel_est(1)
        rx_instance.demode_payload()
        rx_instance.channel_equ(1)
        #rx_instance.ranging()
        
        print("qam_symbs_after_ofdm_demode.real len = ",len(rx_instance.qam_symbs_after_ofdm_demode.real))
        
        print(rx_instance.demode_res_after_channel_equ_1d)
        
        
        with fig_recv_sig.batch_update():
            scatter_recv_sig.data[0].y = rx_instance.recv_sig_after_cutting.real
            scatter_recv_sig.data[0].x = list(range(len(rx_instance.recv_sig_after_cutting.real)))
            fig_recv_sig.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.recv_sig_after_cutting.real) * 1.1])
            )
        
        with fig_time_sync.batch_update():
            scatter_time_sync.data[0].y = rx_instance.time_sync_res_before_cutting
            scatter_time_sync.data[0].x = list(range(len(rx_instance.time_sync_res_before_cutting)))
            fig_time_sync.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
            )
            
        with fig_channel_phase.batch_update():
            scatter_channel_phase.data[0].y = rx_instance.channel_est_phase[0]
            scatter_channel_phase.data[0].x = list(range(len(rx_instance.channel_est_phase[0])))
            fig_channel_phase.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.channel_est_phase[0]) * 1.1])
            )
            
        with fig_channel_gain.batch_update():
            scatter_channel_gain.data[0].y = rx_instance.channel_est_gain[0]
            scatter_channel_gain.data[0].x = list(range(len(rx_instance.channel_est_gain[0])))
            fig_channel_gain.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.channel_est_gain[0]) * 1.1])
        )
            
        with fig_constellation_no_equ.batch_update():
            scatter_constellation_no_equ.data[0].x = rx_instance.qam_symbs_after_ofdm_demode_1d.real
            scatter_constellation_no_equ.data[0].y = rx_instance.qam_symbs_after_ofdm_demode_1d.imag
            
        with fig_constellation_after_equ.batch_update():
            scatter_constellation_after_equ.data[0].x = rx_instance.demode_res_after_channel_equ_1d.real
            scatter_constellation_after_equ.data[0].y = rx_instance.demode_res_after_channel_equ_1d.imag

        '''
        with fig_ranging_res.batch_update():
            scatter_ranging_res.data[0].y = rx_instance.ranging_res[0][0]
            scatter_ranging_res.data[0].x = list(range(len(rx_instance.ranging_res[0][0])))
            fig_ranging_res.update_layout(
                yaxis=dict(range=[0, np.max(rx_instance.ranging_res[0][0]) * 1.1])
            )
        '''
    # 创建输入框
    start_index_input = widgets.Text(value='', description='start_index:', disabled=False)
    end_index_input = widgets.Text(value='', description='end_index:', disabled=False)
    peak_height_input = widgets.Text(value='', description='peak_height:', disabled=False)
    
    # 创建更新按钮
    button = widgets.Button(description='Update and Read')
    button_save_rx_output = widgets.Button(description='save frame')

    # 创建文本输出区域
    output = widgets.Output()

    # 定义按钮点击事件
    def on_button_click(b):
        with output:
            clear_output(wait=True)  # 清空之前的输出
            print(f'你输入的内容是: start_index = {start_index_input.value}, end_index = {end_index_input.value}, peak_height = {peak_height_input.value}')

        start_index_int = int(start_index_input.value)
        end_index_int = int(end_index_input.value)
        peak_height_int = int(float(peak_height_input.value))

        print("start_index_int =", start_index_int, "end_index_int", end_index_int, "peak_height_int", peak_height_int)

        

        update_plots(start_index_int, end_index_int, peak_height_int)
    
    def on_button_click_save_rx_output(b):
        rx0.save_rx_to_file("/home/xilinx/jupyter_notebooks/summer/dats/rx_output/")
        
        with output:
            clear_output(wait=True)  # 清空之前的输出
            print("写入成功")


    button.on_click(on_button_click)
    button_save_rx_output.on_click(on_button_click_save_rx_output)

    # 使用 HBox 组件显示输入框和按钮
    input_box = widgets.HBox([start_index_input, end_index_input, peak_height_input])
    # 使用 HBox 组件显示图表
    hbox0 = widgets.HBox([fig_recv_sig, fig_time_sync])
    hbox1 = widgets.HBox([fig_channel_phase, fig_channel_gain])
    hbox2 = widgets.HBox([fig_constellation_no_equ, fig_constellation_after_equ])
    #hbox3 = widgets.Hbox([fig_ranging_res])
    vbox = widgets.VBox([input_box, button, output, hbox0, hbox1 , hbox2, button_save_rx_output])
    display(vbox)

display_real_time_plots(rx0, update_interval=1)
#scatterplot(rx0.qam_symbs_after_ofdm_demode.real,rx0.qam_symbs_after_ofdm_demode.imag,ax=None)

# 自动更新的星座图
rx_instance.cutting_recv_sig(0,2000,8e20) 需要在这里修改峰的值

In [ ]:
def display_real_time_plots(rx_instance, update_interval=1):
    """
    显示 RX 类实例的 time_sync_res_before_cutting 数组和星座图的实时更新图表。

    参数：
    rx_instance (RX): RX 类的实例
    update_interval (int): 更新间隔时间（秒）
    """

    # 初始化图表
    fig_time_sync = go.FigureWidget()
    scatter_time_sync = fig_time_sync.add_scatter(y=rx_instance.time_sync_res_before_cutting, mode='lines+markers')
    fig_time_sync.update_layout(
        title='Time Sync Results Before Cutting',
        xaxis_title='Index',
        yaxis_title='Value',
        yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
    )

    fig_constellation = go.FigureWidget()
    scatter_constellation = fig_constellation.add_scatter(x=rx_instance.demode_res_after_channel_equ_1d.real, y=rx_instance.demode_res_after_channel_equ_1d.imag, mode='markers')
    fig_constellation.update_layout(
        title='Constellation Diagram',
        xaxis_title='In-Phase',
        yaxis_title='Quadrature',
        xaxis=dict(range=[-100000, 100000], showticklabels=False),
        yaxis=dict(range=[-100000, 100000], showticklabels=False)
    )

    def update_plots():
        for j in range (1):
            dma_opts.rx_dma_transfer_once(rx_I_dma, output_I_buffer)
            dma_opts.rx_dma_transfer_once(rx_Q_dma, output_Q_buffer)
            df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_I_FILE, output_I_buffer[0:2000])
            df.save_array_to_file(ofdm_cfgs.RTL_OUTPUT_Q_FILE, output_Q_buffer[0:2000])

            rx_instance.reload_received_sig()
            rx_instance.first_time_sync()  # 更新数组
            rx_instance.cutting_recv_sig(0,2000,8e20)
            rx_instance.second_time_sync()
            rx_instance.disassemble()
            rx_instance.channel_est(1)
            rx_instance.demode_payload()
            rx_instance.channel_equ(1)

            with fig_time_sync.batch_update():
                scatter_time_sync.data[0].y = rx_instance.time_sync_res_before_cutting
                scatter_time_sync.data[0].x = list(range(len(rx_instance.time_sync_res_before_cutting)))
                fig_time_sync.update_layout(
                    yaxis=dict(range=[0, np.max(rx_instance.time_sync_res_before_cutting) * 1.1])
                )
            with fig_constellation.batch_update():
                scatter_constellation.data[0].x = rx_instance.demode_res_after_channel_equ_1d.real
                scatter_constellation.data[0].y = rx_instance.demode_res_after_channel_equ_1d.imag
            
            #rx_instance.plt_all_qam_constellation()
            #plt.show()
                
                
            #time.sleep(update_interval)

    # 创建输入框
    start_index_input = widgets.Text(value='0', description='start_index:', disabled=False)
    end_index_input = widgets.Text(value='2000', description='end_index:', disabled=False)
    peak_height_input = widgets.Text(value='1.0', description='peak_height:', disabled=False)
    
    # 创建文本输出区域
    output = widgets.Output()

    def update_inputs():
        with output:
            clear_output(wait=True)  # 清空之前的输出
            print(f'你输入的内容是: start_index = {start_index_input.value}, end_index = {end_index_input.value}, peak_height = {peak_height_input.value}')

        start_index_int = int(start_index_input.value)
        end_index_int = int(end_index_input.value)
        peak_height_int = int(float(peak_height_input.value))

        rx_instance.cutting_recv_sig(start_index_int, end_index_int, peak_height_int)

    def periodic_update():
        while True:
            #update_inputs()
            update_plots()
            #time.sleep(update_interval)

    # 使用 HBox 组件显示输入框
    input_box = widgets.HBox([start_index_input, end_index_input, peak_height_input])
    # 使用 HBox 组件显示图表
    hbox = widgets.HBox([fig_time_sync, fig_constellation])
    vbox = widgets.VBox([input_box, hbox, output])
    display(vbox)

    periodic_update()

display_real_time_plots(rx0, update_interval=1)

In [ ]:
rx0.plt_recv_sig()